# Using CAS-VQE-UCC with Multipsi, veloxchem and Qiskit

In this notebook I will show how to get the active space integrals from Multipsi and VeloxChem and transform those into Qbits that can be used in a Quantum Computer. In this case all the quantum part will be carried in IBM Qiskit

In [1]:
import veloxchem as vlx
import multipsi as mtp
import numpy as np
HO2_xyz="""3
O2                                                                                                                         
O    0.000000000000        0.000000000000        0.000000000000
H    0.000000000000        0.740848095288        0.582094932012
H    0.000000000000       -0.740848095288        0.582094932012
"""
print('Available basis sets for oxygen:\n', vlx.MolecularBasis.get_avail_basis('O'))
molecule = vlx.Molecule.from_xyz_string(HO2_xyz)
#basis = ['STO-3G','6-31G','6-31++G**','6-311++G**','DEF2-SVP','DEF2-SVPD']
basis = vlx.MolecularBasis.read(molecule,"6-311++G(2D,2P)")
scfdrv = vlx.ScfRestrictedDriver()
scfdrv.compute(molecule, basis)
print(basis.get_string(molecule))


* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 10.
Available basis sets for oxygen:
 ['6-31++G', '6-31++G*', '6-31++G**', '6-31+G', '6-31+G*', '6-31+G**', '6-311++G', '6-311++G(2D,2P)', '6-311++G(3DF,3PD)', '6-311++G*', '6-311++G**', '6-311+G', '6-311+G(2D,P)', '6-311+G*', '6-311+G**', '6-311G', '6-311G(2DF,2PD)', '6-311G*', '6-311G**', '6-31G', '6-31G(2DF,P)', '6-31G(3DF,3PD)', '6-31G*', '6-31G**', 'ANO-L-MB', 'ANO-L-VDZP', 'ANO-L-VQZP', 'ANO-L-VTZP', 'ANO-S-MB', 'ANO-S-VDZP', 'AO-START-GUESS', 'AUG-CC-PCV5Z', 'AUG-CC-PCVDZ', 'AUG-CC-PCVQZ', 'AUG-CC-PCVTZ', 'AUG-CC-PV5Z', 'AUG-CC-PV6Z', 'AUG-CC-PVDZ', 'AUG-CC-PVQZ', 'AUG-CC-PVTZ', 'AUG-PCSEG-0', 'AUG-PCSEG-1', 'AUG-PCSEG-2', 'AUG-PCSEG-3', 'AUG-PCSEG-4', 'AUG-PCX-1', 'AUG-PCX-2', 'AUG-PCX-3', 'AUG-PCX-4', 'CC-PCV5Z', 'CC-PCVDZ', 'CC-PCVQZ', 'CC-PCVTZ', 'CC-PV5Z', 'CC-PV6Z', 'CC-PVDZ', 'CC-PVQZ', 'CC-PVTZ', 'DEF2-QZVP', 'DEF2-QZVPD', 'DEF2-QZVPP', 'DEF2-QZVPPD', 'DEF2-SV(P)', 'DEF2-SVP'

In [57]:
space=mtp.OrbSpace(molecule,scfdrv.mol_orbs)
space.CAS(2,2)
expansion = mtp.CIExpansion(space)
print(expansion)
print(space.norb)


          CI expansion:
          -------------
Number of determinants:      3


47


In [58]:
CASham = mtp.CIOperator(molecule=molecule,basis=basis,expansion=expansion)
CASham.compute_ints()

In [59]:
h1 = CASham.Ftu ## One electron integrals in MO
h2 = CASham.tuvw ## Two electron integrals in MO

In [60]:
CIdrv=mtp.CIDriver(molecule,basis,space)
#By default, the guess is done by diagonalising a small segment of the full Hamiltonian
#here set the dimension of this Hamiltonian to 1, which corresponds to simply starting from a single SD
CIdrv.nguess=1
CIdrv.compute(1)



          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 4
Number of active orbitals:              2
Number of virtual orbitals:             41

    This is a CASSCF wavefunction: CAS(2,2)

          CI expansion:
          -------------
Number of determinants:      3



        CI Iterations
        -------------

Iteration   Average energy      Time
     1        -76.05664557   0:00:00
     2        -76.05665027   0:00:00
     3        -76.05665027   0:00:00
Convergence reached in 3 iterations

        Final results
        -------------

* State 1
- Energy: -76.05665027236938
- S^2   : 0.00  (multiplicity = 1.0 )
- Natural orbitals
[1.99999 0.00001]


In [61]:
from qiskit_nature.properties.second_quantization.electronic.integrals import (OneBodyElectronicIntegrals,
TwoBodyElectronicIntegrals,)
from qiskit_nature.properties.second_quantization.electronic.electronic_energy import ElectronicEnergy
from qiskit_nature.properties.second_quantization.electronic.bases import ElectronicBasis

In [62]:
h1_so = OneBodyElectronicIntegrals(ElectronicBasis.MO,(h1,h1)).to_spin()
H1 = OneBodyElectronicIntegrals(ElectronicBasis.SO,h1_so,)
h2_so = TwoBodyElectronicIntegrals(ElectronicBasis.MO,(h2,h2,h2,h2)).to_spin()
H2 = TwoBodyElectronicIntegrals(ElectronicBasis.SO,h2_so,)

In [63]:
electronic_energy = ElectronicEnergy([H1,H2])
print(electronic_energy)

ElectronicEnergy
	(SO) 1-Body Terms:
		<(4, 4) matrix with 4 non-zero entries>
		[0, 0] = -1.2502132025117152
		[1, 1] = -0.35246033620532763
		[2, 2] = -1.2502132025117152
		[3, 3] = -0.35246033620532763
	(SO) 2-Body Terms:
		<(4, 4, 4, 4) matrix with 32 non-zero entries>
		[0, 0, 0, 0] = -0.36943378871397536
		[0, 0, 1, 1] = -0.0012000949834511912
		[0, 1, 0, 1] = -0.0012000949834511912
		[0, 1, 1, 0] = -0.09972804073497063
		[0, 2, 2, 0] = -0.36943378871397536
		... skipping 27 entries


In [64]:
hamiltonian = electronic_energy.second_q_ops()[0]
print(hamiltonian)

Fermionic Operator
register length=4, number terms=14
  (0.0024001899669023824+0j) * ( +_0 -_1 +_2 -_3 )
+ (-0.002400189966902379+0j) * ( +_0 -_1 -_2 +_3 )
+ (-0.002400189966902379+0j) * ( -_0 +_1 +_2 -_3 )
+ (0.002400189966902374+0j) * ( -_0 +_1 -_2 ...


In [65]:
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import BravyiKitaevMapper

BK_conv = QubitConverter(BravyiKitaevMapper())

In [66]:
qbit_op = BK_conv.convert(hamiltonian)
print(qbit_op)

0.0006000474917255946 * ZXIX
- 0.0006000474917255946 * IXZX
- 0.0006000474917255946 * ZXZX
+ 0.0006000474917255946 * IXIX
- 1.177259546538304 * IIII
+ 0.03466106352415646 * ZZZI
+ 0.34126171365562474 * IZII
+ 0.049263972875759746 * ZIZI
+ 0.03466106352415657 * IIZZ
+ 0.04244111133526223 * ZZIZ
+ 0.04986402036748534 * IZZZ
+ 0.3412617136556247 * IIIZ
+ 0.04986402036748534 * ZZZZ
+ 0.18471689435698768 * IZIZ
+ 0.049263972875759746 * IIZI


In [73]:
from qiskit.algorithms import VQE
from qiskit_nature.circuit.library import HartreeFock
from qiskit_nature.circuit.library import UCCSD 
num_orbitals= 4
num_particles =(1,1)
init_state = HartreeFock(num_orbitals,num_particles,BK_conv)
ansatz = UCCSD(BK_conv,num_particles,num_orbitals,reps=10,initial_state=init_state)
#ansatz.decompose().draw('mpl',style='iqx')

In [74]:
from qiskit.providers.aer import StatevectorSimulator
backend = StatevectorSimulator()
from qiskit.algorithms.optimizers import SLSQP
algorithm = VQE(ansatz,SLSQP(maxiter=1e6,tol=1e-12),quantum_instance=backend)
result = algorithm.compute_minimum_eigenvalue(qbit_op)
print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 64,
    'eigenstate': array([ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
       -1.95708511e-03+3.93721499e-17j, -5.44540207e-09-8.40880959e-16j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        2.49086924e-08-1.76647385e-15j,  9.99998085e-01-1.13437418e-14j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j]),
    'eigenvalue': (-1.7615635249842598+0j),
    'optimal_parameters': {   ParameterVectorElement(t[8]): 0.00019570304681034808,
                              ParameterVectorElement(t[3]): -3.6450897626230104e-09,
                              ParameterVectorElement(t[7]): 4.252604723060178e-09,
                              P

In [69]:
def cas_energy(result):
    'Uses the eigenvalue from the VQE to compute the CAS-SCF energy'
    print ('Inactive space contribution in a.u.:',float(CIdrv.ham.inEne))
    print ('VQE energy for the active space in a.u.:',result.eigenvalue.real)
    print ('Nuclear repulsion in a.u.:',molecule.nuclear_repulsion_energy())
    print ('CASSCF-VQE energy in a.u.:',float(CIdrv.ham.inEne)+result.eigenvalue.real)
    casscf_vqe = float(CIdrv.ham.inEne)+result.eigenvalue.real
    return casscf_vqe

In [70]:
casscf_vqe = cas_energy(result)

Inactive space contribution in a.u.: -74.29508674738514
VQE energy for the active space in a.u.: -1.7615635249842598
Nuclear repulsion in a.u.: 9.343638157670377
CASSCF-VQE energy in a.u.: -76.0566502723694


In [71]:
multipsi_cas = -76.05665027236938
comparision_vqe_ve = casscf_vqe - multipsi_cas
print(comparision_vqe_ve*627.51,'kcal/mol')


-8.91745344233641e-12 kcal/mol
